#### 세금 관련 챗봇 모델 생성<hr>
1. 데이터 전처리 (분류 모델을 통해)
2. 모델 학습

1. 데이터 전처리<hr>

In [1]:
#패키지 불러오기
import pandas as pd
from transformers import pipeline
# from sentence_split import split_answer
import os
from kss import split_sentences


ModuleNotFoundError: No module named 'transformers'

In [ ]:
# 데이터 경로 폴더 지정
DATA_PATH='./test_1.csv'
dataDF=pd.read_csv(DATA_PATH)


In [ ]:
# def split_answer(DataFrame, save_name, save_path):

dataDF=dataDF
splited_answer=[]
# 행마다 문장분리 실행
for a in dataDF['answer']:
    # a= a.replace('\u200b', '')
    a_splited=split_sentences(a)
    # 구두점 넣기
    sentence_list=[]
    # print(a_splited, type(a_splited))
    for sentence in a_splited:
        sentence=str(sentence)
        # print(sentence, type(sentence))
        if sentence[-1] not in ['.', '!', '?', '~', ')', '}', ']',
                                '&', '>']:
            sentence_list.append(sentence+'.')
            # print(sentence)
        else:
            sentence_list.append(sentence)
            # print('구두점 존재')
    # 분리한 행 재조합
    
    splited_answer.append(sentence_list)
# 데이터 프레임으로 반환
dataDF['answer']=splited_answer
# splited로 이름 바꾸기
# data=data.replace('after', 'splited')
dataDF.to_csv('./test_splited.csv', index=False, encoding='utf-8-sig')

In [ ]:
# 문장 분류기로 필요없는 문장 제거
splitDF=pd.read_csv('./split_data.csv')
#파이프라인 생성
classifier = pipeline(
    task="text-classification",
    model="./model_CLF_2/",
    tokenizer="./model_CLF_2/", 
    device='cuda'
)
# 문장마다 분류기 실행
cleaned_text_lists=[]
for row in splitDF['answer']:
    row=[]
    for sentence in row: 
        print(sentence)
        results = classifier(sentence)
        # 결과에 따른 전처리 처리
        if result['label']== 'LABEL_0':
            texts.remove(text)
        else: row.append(text)
    # 전처리한 행 하나의 string으로 재조합
    cleaned_text_lists.append(row)
splitDF['answer']=cleaned_text_lists


You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [ ]:
splitDF=pd.read_csv('./split_data.csv')
type(list(splitDF['answer'])[0])
# for row in splitDF['answer']:
#     new_row=[]
#     print(row[1])
#     break

str

In [ ]:
# 필요없는 문장 제거 하기
import torch
from transformers import T5ForSequenceClassification, T5Tokenizer
DEVICE='cuda' if torch.cuda.is_available() else 'cpu'
# 분류 모델 지정하기
MODEL_NAME='./model_CLF_2'
tokenizer=T5Tokenizer.from_pretrained(MODEL_NAME)
model= T5ForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2).to(DEVICE)


# 예측하기 (테스트)
def predict_text(text):
    text=text
    inputs = tokenizer(f"classify: {text}", return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs.to(DEVICE))
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_class = predictions.argmax().item()
    return predicted_class, predictions[0].tolist()

text = "정말 좋은 영화였어요"
predicted_class, probabilities = predict_text(text)
print(f"예측 클래스: {predicted_class}")
print(f"클래스별 확률: {probabilities}")

예측 클래스: 0
클래스별 확률: [0.9609748721122742, 0.03902510553598404]


In [ ]:
# 문장 문류를 통한 데이터 전처리 수행
import re
new_rows=[]
for row in splitDF['answer']:
    # 콤마 기준으로 문장 분리
    sentence_list = re.split(r'[,]', row)
    for sentence in sentence_list:
        # 문장당 분류 모델 실행
        result, _=predict_text(sentence)
        if result==0:
            sentence_list.remove(sentence)
        else: pass
    new_rows.append(sentence_list)

In [ ]:
# 열에 적용
splitDF['answer']=new_rows

In [ ]:
splitDF.head()

,Unnamed: 0,question,answer
0,0,제가 일반사업자 및 직장인인데 이번에 빌라를 팔면 소득이 생길거같은데 소득세와종합소...,"[ '저의 답변이 도움이 되기를 바랍니다.', '1. 일반사업자 및 직장인인데 빌..."
1,1,"종합소득세는 개인이 1년간 벌어들인 소득에 대해 부과되는 세금으로, 과세표준에 따라...","[['종합소득세율은 다음과 같습니다.', '국세청 사이트https://www.nt..."
2,2,"종합소득세는 개인이 1년간 벌어들인 소득에 대해 부과되는 세금으로, 과세표준에 따라...","[ '종합소득세 세율은 누진세율(6%~45%)이 적용되며, ""과세표준에 따라 적용..."
3,3,"종합소득세는 개인이 1년간 벌어들인 소득에 대해 부과되는 세금으로, 과세표준에 따라...",[ '5월에는 종합소득세 납부 기간으로 사업하시는 분들이나 프리...blog.nav...
4,4,"종합소득세는 개인이 1년간 벌어들인 소득에 대해 부과되는 세금으로, 과세표준에 따라...","[ '아래 링크에서 볼 수 있습니다.', 'https://www.nts.go.kr..."


2. 데이터셋 형성<hr>

In [ ]:
from datasets import Dataset, load_dataset

In [ ]:
# 데이터셋 분리

textDF=splitDF[['question', 'answer']].reset_index(drop=True)
train_length= int(len(textDF)*0.8)
valid_length=int(len(textDF)*0.15)
test_length=int(len(textDF)-train_length-valid_length)
print(train_length, valid_length, test_length)
trainDF=textDF[:train_length]
validDF=textDF.drop(trainDF.index).reset_index(drop=True)[:valid_length]
testDF=textDF.drop(trainDF.index).reset_index(drop=True)[valid_length:]
len(trainDF), len(validDF), len(testDF)

5028 942 316


(5028, 942, 316)

In [ ]:
new_rows=[]
for row in trainDF['answer']:
    result = '\n'.join(eval(row))
    new_rows.append(result)
trainDF['answer']=new_rows
new_rows=[]
for row in validDF['answer']:
    result = '\n'.join(eval(row))
    new_rows.append(result)
validDF['answer']=new_rows
new_rows=[]
for row in testDF['answer']:
    result = '\n'.join(eval(row))
    new_rows.append(result)
testDF['answer']=new_rows

TypeError: eval() arg 1 must be a string, bytes or code object

str

In [ ]:
# 저장
trainDF.to_csv('./train_3.csv', index=False)
validDF.to_csv('./valid_3.csv', index=False)
testDF.to_csv('./test_3.csv', index=False)

In [ ]:
data_dicts= {"train": "train_3.csv", "valid": "valid_3.csv", "test": "test_3.csv"}
# 데이터셋 생성
dataset= load_dataset('csv' ,data_files=data_dicts)

### 모델 학습<hr>

In [ ]:
import torch
from transformers import T5TokenizerFast, T5ForConditionalGeneration
# 사전학습된 모델 불러오기
BASE='KETI-AIR/ke-t5-base-ko'
tokenizer=T5TokenizerFast.from_pretrained(BASE)
model= T5ForConditionalGeneration.from_pretrained(BASE)
# 쿠다 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model=model.to(device)
# 문장 최대 길이 설정
max_length=200

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
# 토큰화 함수
def make_tokenized(test):
    tokenized= tokenizer(test['question'],
                         text_target=test['answer'],
                         max_length=200,
                         truncation=True)
    return tokenized

In [ ]:
# 토큰화 적용
tokenizedDS=dataset.map(make_tokenized,
batched=True,
remove_columns=dataset['train'].column_names
)

Map:   0%|          | 0/316 [00:00<?, ? examples/s]

In [ ]:
tokenizedDS

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5028
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 942
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 316
    })
})

In [ ]:
# 토큰화 확인
print( '원 데이터    :', dataset['train'][10]['question'] )
print( '처리 후 데이터:', tokenizedDS['train'][10]['input_ids'] )
print( '토큰화       :', tokenizer.convert_ids_to_tokens(tokenizedDS['train'][10]['input_ids']) )

print('\n')
print( '원 데이터    :', dataset['train'][10]['answer'] )
print( '토큰화       :', tokenizer.convert_ids_to_tokens(tokenizedDS['train'][10]['labels']) )
print( '처리 후 데이터:', tokenizedDS['train'][10]['labels'] )

원 데이터    : 저는 직장을 다니고 있고 저희 어머니께서 제 명의로 사업자를 내셔서 일반음식점 자영업을 하고 계십니다.이경우 발생하는 종합소득세는 사업자에서 발생한 세금일지 아니면 직장소득과 사업소득이 합쳐져 생긴 세금일지 궁금합니다.직장에서 연말정산 등등 모든 세금처리를 다하는데 종합소득세는 정확히 무엇인지가 궁금해요.
처리 후 데이터: [1928, 31404, 33541, 561, 6460, 4663, 6225, 190, 21835, 4746, 21, 182, 24932, 1039, 41650, 38176, 11, 239, 55996, 3, 15, 508, 363, 5127, 1902, 33642, 12, 4746, 37, 2303, 6572, 30, 76, 3533, 5660, 5130, 31, 546, 5130, 15, 9776, 1781, 6321, 6572, 30, 76, 17282, 1008, 3, 25508, 37, 38097, 27923, 350, 6572, 4582, 21, 299, 2143, 1902, 33642, 12, 6919, 9118, 22, 16271, 328, 3, 1]
토큰화       : ['▁저는', '▁직장을', '▁다니고', '▁있고', '▁저희', '▁어머니', '께서', '▁제', '▁명의로', '▁사업자', '를', '▁내', '셔서', '▁일반', '음식점', '▁자영업', '을', '▁하고', '▁계십니다', '.', '이', '경', '우', '▁발생하는', '▁종합', '소득세', '는', '▁사업자', '에서', '▁발생한', '▁세금', '일', '지', '▁아니면', '▁직장', '소득', '과', '▁사업', '소득', '이', '▁합쳐', '져', '▁생긴', '▁세금', '일', '지', '▁궁금', '합니다', '.', '직장', '에서', '▁연말정산', '▁등등', '▁모든', '▁세금', '처리', '를', '▁다', '하는데', '▁종합', '소득세', '는', '▁정확히', '▁무엇인

In [ ]:
# 인코더 입력
encoder_inputs = tokenizer(
    ["사장님이 간이사업자라 4대보험은 내년부터 되고원청징수 3.3프로 신청을 홈택스로 해주셨는데원래 제 개인정보가 안들어 가고 사장님 사업소득신고러 되나요? 아니면 제가 추가적으로 근로소득을 신청해야하나요?"],
    return_tensors="pt"
)['input_ids'].to(device)
# 디코더 입력을 위해 넣고
decoder_targets = tokenizer(
    ["1.3.3%사업소득원천징수시 주민등록번호가 키코드입니다. 신분증사본을 제출해야 사업주가  사업소득신고를 할 수 있습니다.2.사업소득신고는 사업자가 해야합니다. 귀하가 근로소득으로 신고할 수 없습니다.', '물음표 둘다 안됩니다."],
    return_tensors="pt"
)['input_ids'].to(device)
# 디코더 쪽으로 이동
decoder_inputs = model._shift_right(decoder_targets)

pd.DataFrame(
    [
        tokenizer.convert_ids_to_tokens(decoder_targets[0]),
        tokenizer.convert_ids_to_tokens(decoder_inputs[0])
    ],
    index=('decoder target', 'decoder input')
)

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
decoder target,▁1,.,3.3%,사업,소득,원,천,징,수,시,...,▁',물,음,표,▁둘,다,▁안,됩니다,.,</s>
decoder input,<pad>,▁1,.,3.3%,사업,소득,원,천,징,수,...,",",▁',물,음,표,▁둘,다,▁안,됩니다,.


In [ ]:
outputs = model(input_ids=encoder_inputs,
                decoder_input_ids=decoder_inputs,
                labels=decoder_targets)
outputs['encoder_last_hidden_state'].shape

torch.Size([1, 49, 768])

In [ ]:
# 콜레이터 사용
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
trainDL=tokenizedDS['train']

collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, 
                                model=model,
                                return_tensors="pt"
                                )
batch = collator(
    [tokenizedDS["train"][i] for i in range(1, 3)]
)

In [ ]:
import numpy as np
import nltk
# BLEU 점수 측정
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # decode preds and labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # rougeLSum expects newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return result

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    
    # 토큰을 텍스트로 디코딩
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # BLEU 점수 계산
    bleu_score = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    
    return {"bleu": bleu_score["bleu"]}

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="chkpt",
    learning_rate=0.0005,
    weight_decay=0.01,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=10,
    save_steps=500,
    save_total_limit=2,
    eval_strategy="steps",
    logging_strategy="no",
    predict_with_generate=True,
    fp16=False,
    gradient_accumulation_steps=3,
    report_to="none" # Wandb 로그 끄기
)

TypeError: __init__() got an unexpected keyword argument 'compute_metrics'

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=tokenizedDS["train"],
    eval_dataset=tokenizedDS["valid"],
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,

)
# trainer.train()
# trainer.save_model("./results_3")

------------------------

In [ ]:
import evaluate
# 평가지표 테스트 (rouge)
eval=evaluate.load('rouge')

In [ ]:
trainer.evaluate()


c:\Users\aukii\anaconda3\envs\nlp_38\lib\site-packages\transformers\generation\utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


  0%|          | 0/942 [00:00<?, ?it/s]

OverflowError: can't convert negative int to unsigned